In [22]:
import numpy as np
import pandas as pd

class AmericanOption:
    def __init__(self, S0, K, r, sigma, T, N, option_type='C'):
        self.S0 = S0
        self.K = K
        self.r = r
        self.sigma = sigma
        self.T = T
        self.N = N
        self.option_type = option_type
        self.pricing_strategy = None

    def set_pricing_strategy(self, strategy):
        self.pricing_strategy = strategy

    def price(self):
        return self.pricing_strategy.price(self.S0, self.K, self.r, self.sigma, self.T, self.N, self.option_type)

    def greeks(self):
        return self.pricing_strategy.greeks(self.S0, self.K, self.r, self.sigma, self.T, self.N, self.option_type)
    
class TrigeorgisPricing:
    def price(self, S0, K, r, sigma, T, N, opttype):
        dt = T/N
        nu = r - 0.5*sigma**2
        dxu = np.sqrt(sigma**2 * dt + nu**2 * dt**2)
        dxd = -dxu
        pu = 0.5 + 0.5*nu*dt/dxu
        pd = 1-pu
        disc = np.exp(-r*dt)
        S = np.zeros(N+1)
        S[0] = S0*np.exp(N*dxd)
        for j in range(1, N+1):
            S[j] = S[j-1]*np.exp(dxu - dxd)
        C = np.zeros(N+1)
        for j in range(N+1):
            if opttype == 'C':
                C[j] = max(0, S[j]-K)
            else:
                C[j] = max(0, K - S[j])
        for i in np.arange(N, 0, -1):
            for j in range(i):
                C[j] = disc * (pu*C[j+1] + pd*C[j])
        return C[0]
    
    def greeks(self, S0, K, r, sigma, T, N, opttype):
        # Delta and Gamma as before
        dS = 0.01 * S0
        price1 = self.price(S0 + dS, K, r, sigma, T, N, opttype)
        price2 = self.price(S0 - dS, K, r, sigma, T, N, opttype)
        delta = (price1 - price2) / (2 * dS)
        delta1 = (price1 - self.price(S0, K, r, sigma, T, N, opttype)) / dS
        delta2 = (self.price(S0, K, r, sigma, T, N, opttype) - price2) / dS
        gamma = (delta1 - delta2) / dS

        # Theta
        dt = 0.01 * T
        theta = (self.price(S0, K, r, sigma, T + dt, N, opttype) - self.price(S0, K, r, sigma, T, N, opttype)) / dt

        # Vega
        dsigma = 0.01 * sigma
        vega = (self.price(S0, K, r, sigma + dsigma, T, N, opttype) - self.price(S0, K, r, sigma, T, N, opttype)) / dsigma
        
        # Rho
        dr = 0.01 * r 
        rho = (self.price(S0, K, r + dr, sigma, T, N, opttype) - self.price(S0, K, r, sigma, T, N, opttype)) / dr

        return delta, gamma, theta, vega, rho
    
#Example Usage
option = AmericanOption(S0=100, K=100, r=0.05, sigma=0.2, T=1, N=100, option_type='C')
trigeorgis_pricing = TrigeorgisPricing()
option.set_pricing_strategy(trigeorgis_pricing)

# Compute option price and Greeks
price = option.price()
delta, gamma, theta, vega, rho = option.greeks()


# Function to compute and add to DataFrame
def compute_and_add_to_df(option, df):
    price = option.price()
    delta, gamma, theta, vega, rho = option.greeks()
    
    new_data = {
        "Option Type": [option.option_type],
        "Price": [price],
        "Delta": [delta],
        "Gamma": [gamma],
        "Theta": [theta],
        "Vega": [vega],
        "Rho" : [rho], 
        "S0": [option.S0],
        "K": [option.K],
        "r": [option.r],
        "sigma": [option.sigma],
        "T": [option.T],
        "N": [option.N]
    }
    
    return df.vstack(pl.DataFrame(new_data))

# Initialize DataFrame with correct data types
def compute_and_add_to_df(option, df):
    price = option.price()
    delta, gamma, theta, vega, rho = option.greeks()
    
    new_data = {
        "Option Type": option.option_type,
        "Price": price,
        "Delta": delta,
        "Gamma": gamma,
        "Theta": theta,
        "Vega": vega,
        "Rho" : rho,
        "S0": option.S0,
        "K": option.K,
        "r": option.r,
        "sigma": option.sigma,
        "T": option.T,
        "N": option.N
    }
    
    return df.append(new_data, ignore_index=True)

# Initialize DataFrame
df = pd.DataFrame(columns=["Option Type", "Price", "Delta", "Gamma", "Theta", "Vega", "Rho", "S0", "K", "r", "sigma", "T", "N"])

# Compute for Call
call_option = AmericanOption(S0=100, K=100, r=0.05, sigma=0.2, T=1, N=100, option_type='C')
call_option.set_pricing_strategy(trigeorgis_pricing)
df = compute_and_add_to_df(call_option, df)

# Compute for Put
put_option = AmericanOption(S0=100, K=100, r=0.05, sigma=0.2, T=1, N=100, option_type='P')
put_option.set_pricing_strategy(trigeorgis_pricing)
df = compute_and_add_to_df(put_option, df)

print(df)

  Option Type      Price     Delta     Gamma     Theta       Vega        Rho  \
0           C  10.431114  0.636492  0.074861  6.394110  37.422284  53.299664   
1           P   5.554590 -0.363502  0.074861  1.640223  37.426276 -41.786175   

    S0    K     r  sigma  T    N  
0  100  100  0.05    0.2  1  100  
1  100  100  0.05    0.2  1  100  


/var/folders/jq/zct5m03s7dzgrm26f845lhjr0000gn/T/ipykernel_5069/4210383828.py:126: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return df.append(new_data, ignore_index=True)
/var/folders/jq/zct5m03s7dzgrm26f845lhjr0000gn/T/ipykernel_5069/4210383828.py:126: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return df.append(new_data, ignore_index=True)
